In [1]:
import read_write_model as colmap_reader
import numpy as np
import torch
from torch import nn
import imageio.v3 as iio
import matplotlib.pyplot as plt
from datautils import read_data
from nerfhelpers import get_rays, batched_get_rays, NeRF
%load_ext autoreload
%autoreload 2

In [100]:
imagedir = './images'
camFile = 'cameras.bin'
imgFile = 'images.bin'

images, c2w, W, H, F = read_data(camFile, imgFile, imagedir)
images = torch.tensor(images, dtype = torch.float32)
c2w = torch.tensor(c2w, dtype = torch.float32)

Loading Poses...
Done
Reading Images...


KeyboardInterrupt: 

In [4]:
rays_o, rays_d = get_rays(H, W, F, c2w[0])

In [250]:
rays_o = torch.rand((5, 5, 3))
rays_d = torch.rand((5, 5, 3))

image = torch.rand((5, 5, 3))

model = NeRF()
t = model.forward(rays_o, rays_d)

W = 5
H = 5

N_samples = 20

start = 0.
stop = 10.

def render(H, W, start, stop, N_samples, model):
    samples = torch.linspace(start, stop, N_samples) + torch.rand(list(rays_o.shape[:-1]) + [N_samples]) * (stop-start)/N_samples
    xyz = (samples[..., :, None] * rays_d[..., None, :] + rays_o[..., None, :])
    views = rays_d[..., None, :].tile(1, 1, N_samples, 1)
    
    rgb, alpha = model.forward(xyz.reshape(-1, 3), views.reshape(-1, 3))
    
    #Get distances, and append 1e10 to the end to simulate infinitely extending ray
    dists = torch.cat([samples[..., 1:] - samples[..., :-1], torch.tensor([1e10]).expand(samples[...,:1].shape)], -1).reshape(-1, 1)
    alpha_i = 1 - torch.exp(-dists * alpha)
    weights = alpha_i * torch.cumprod(1-alpha + 1e-10, -1) / (1-alpha + 1e-10)[0]
    
    rgb_vals = torch.sum((rgb * weights).reshape(H*W, N_samples, -1), -2)
    depth_map = torch.sum((weights * samples.reshape(-1, 1)).reshape(H*W, N_samples, -1), -2)
    acc_map = torch.sum(weights.reshape(H*W, N_samples, -1), -2)

    return rgb_vals, depth_map, acc_map

In [251]:
alpha.mean()

tensor(0.0434, grad_fn=<MeanBackward0>)

In [252]:
#Get distances, and append 1e10 to the end to simulate infinitely extending ray
dists = torch.cat([samples[..., 1:] - samples[..., :-1], torch.tensor([1e10]).expand(samples[...,:1].shape)], -1).reshape(-1, 1)
alpha_i = 1 - torch.exp(-dists * alpha)
weights = alpha_i * torch.cumprod(1-alpha + 1e-10, -1) / (1-alpha + 1e-10)[0]

rgb_vals = torch.sum((rgb * weights).reshape(H*W, N_samples, -1), -2)
depth_map = torch.sum((weights * samples.reshape(-1, 1)).reshape(H*W, N_samples, -1), -2)
acc_map = torch.sum(weights.reshape(H*W, N_samples, -1), -2)

In [242]:
((rgb * weights).reshape(H*W, N_samples, -1)).shape

torch.Size([25, 20, 3])

In [229]:
rgb.shape

torch.Size([500, 3])

tensor([[2.2030e-03],
        [9.8029e-03],
        [4.3200e-03],
        [1.8694e-02],
        [1.8258e-02],
        [2.5296e-02],
        [3.7822e-02],
        [1.6431e-02],
        [3.8354e-02],
        [5.1406e-02],
        [7.0350e-02],
        [4.2608e-02],
        [2.3079e-02],
        [9.9697e-02],
        [5.9632e-02],
        [7.8396e-02],
        [3.8885e-02],
        [1.3985e-01],
        [2.1743e-02],
        [1.0102e+01],
        [9.8777e-04],
        [2.3673e-03],
        [1.2941e-02],
        [2.0404e-02],
        [1.2437e-02],
        [2.1080e-02],
        [3.1068e-02],
        [3.2030e-02],
        [6.3940e-02],
        [2.2150e-02],
        [5.3254e-02],
        [5.7757e-02],
        [5.2269e-02],
        [8.4726e-02],
        [5.2271e-02],
        [3.0483e-02],
        [1.0493e-01],
        [8.8102e-02],
        [4.3063e-02],
        [1.0060e+01],
        [1.4180e-03],
        [3.1737e-03],
        [1.0920e-02],
        [1.5968e-02],
        [1.5114e-02],
        [2

In [220]:
samples.shape

torch.Size([5, 5, 20])

In [201]:
weights.shape

torch.Size([500, 1])

In [181]:
T_i = 1 - torch.exp(dists * alpha)

tensor(0.0583, grad_fn=<MeanBackward0>)

torch.Size([500, 1])

In [141]:
dists.shape

torch.Size([5, 5, 20])